In [1]:
text = "hello world"
print(text.upper())  # ----> upper() is a method of the string object


HELLO WORLD


In [9]:
import sqlite3
import re

In [10]:
conn = sqlite3.connect('words.db')
cursor = conn.cursor()

In [11]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS words (
    word TEXT NOT NULL,
    definition TEXT,
    example TEXT,
    synonyms TEXT
)
''')

In [12]:
#reading and understanding the data

with open ('/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/SARAS/Saras_-Smart-Define/DataBase/dict/data.adj','r') as file:
    for index, line in enumerate(file):
        if index >= 90 and index <= 300:
            break
        print(line)

  1 This software and database is being provided to you, the LICENSEE, by  

  2 Princeton University under the following license.  By obtaining, using  

  3 and/or copying this software and database, you agree that you have  

  4 read, understood, and will comply with these terms and conditions.:  

  5   

  6 Permission to use, copy, modify and distribute this software and  

  7 database and its documentation for any purpose and without fee or  

  8 royalty is hereby granted, provided that you agree to comply with  

  9 the following copyright notice and statements, including the disclaimer,  

  10 and that the same appear on ALL copies of the software, database and  

  11 documentation, including modifications that you make for internal  

  12 use or for distribution.  

  13   

  14 WordNet 3.1 Copyright 2011 by Princeton University.  All rights reserved.  

  15   

  16 THIS SOFTWARE AND DATABASE IS PROVIDED "AS IS" AND PRINCETON  

  17 UNIVERSITY MAKES NO REPRESENTATI

In [13]:
import re

#defining a function, to read lines in the file and organise them as WORD, DEFINITION, EXAMPLE, SYNONYMS
def parse_file(filepath):
    words_data = []  # List to store cleaned word entries

    with open(filepath, 'r') as file:         #Opening the file in read mode
        for line in file:
            parts = line.split("|")
            if len(parts) < 2:
                continue
            
            # Extract words (main + synonyms)
            first_part = parts[0].strip().split()
            valid_words = [word for word in first_part if word.isalpha() and len(word) > 1]

            
            if not valid_words:
                continue  # Skip lines where no valid words are found
            
            # Main word is the first word
            main_word = valid_words[0]
            synonyms = valid_words[1:]  # Everything else is a synonym

            # Extract definition and example
            definition = parts[1].strip()
            example = parts[2].strip() if len(parts) > 2 else None

            # Store the processed data
            words_data.append({
                "word": main_word,
                "definition": definition,
                "example": example,
                "synonyms": synonyms
            })

    return words_data 

In [14]:
file_path = "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/SARAS/Saras_-Smart-Define/DataBase/dict/data.adj"
word_entries = parse_file(file_path)
word_entries

[{'word': 'able',
  'definition': '(usually followed by `to\') having the necessary means or skill or know-how or authority to do something; "able to swim"; "she was able to program her computer"; "we were at last able to buy a car"; "able to get a grant for the project"',
  'example': None,
  'synonyms': []},
 {'word': 'unable',
  'definition': '(usually followed by `to\') not having the necessary means or skill or know-how; "unable to get to town without a car"; "unable to obtain funds"',
  'example': None,
  'synonyms': []},
 {'word': 'abaxial',
  'definition': 'facing away from the axis of an organ or organism; "the abaxial surface of a leaf is the underside or side facing away from the stem"',
  'example': None,
  'synonyms': ['dorsal']},
 {'word': 'adaxial',
  'definition': 'nearest to or facing toward the axis of an organ or organism; "the upper side of a leaf is known as the adaxial surface"',
  'example': None,
  'synonyms': ['ventral']},
 {'word': 'acroscopic',
  'definition'

In [15]:
# List of file paths
file_paths = ["/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/SARAS/Saras_-Smart-Define/DataBase/dict/data.verb",
              "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/SARAS/Saras_-Smart-Define/DataBase/dict/data.noun",
              "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/SARAS/Saras_-Smart-Define/DataBase/dict/data.adj",
              "/Users/surisettivamsikrishna/Downloads/Vamsi Pc/CODES/Mark2/SARAS/Saras_-Smart-Define/DataBase/dict/data.adv"
              ]  

# Collecting word entries from all files
all_word_entries = []
for path in file_paths:
    all_word_entries.extend(parse_file(path))  # Merging all parsed data into one list

# Now, all_word_entries contains words from all files in a single output

# Let's print the first 5 entries
all_word_entries[:500]


[{'word': 'breathe',
  'definition': 'draw air into, and expel out of, the lungs; "I can breathe better when the air is clean"; "The patient is respiring"',
  'example': None,
  'synonyms': ['respire', 'suspire']},
 {'word': 'respire',
  'definition': 'undergo the biomedical and metabolic processes of respiration by taking up oxygen and producing carbon monoxide',
  'example': None,
  'synonyms': []},
 {'word': 'respire',
  'definition': 'breathe easily again, as after exertion or anxiety',
  'example': None,
  'synonyms': []},
 {'word': 'choke',
  'definition': 'breathe with great difficulty, as when experiencing a strong emotion; "She choked with emotion when she spoke about her deceased husband"',
  'example': None,
  'synonyms': []},
 {'word': 'hyperventilate',
  'definition': 'breathe excessively hard and fast; "The mountain climber started to hyperventilate"',
  'example': None,
  'synonyms': []},
 {'word': 'hyperventilate',
  'definition': 'produce hyperventilation in; "The nurs

In [16]:
len(all_word_entries)

89500

In [17]:
def insert_data_into_db(entries):
    for entry in entries:
        cursor.execute('''
        INSERT INTO words (word, definition, example, synonyms)
        VALUES (?, ?, ?, ?)
        ''', (entry["word"], entry["definition"], entry["example"], ", ".join(entry["synonyms"]) if entry["synonyms"] else None))  
        # ---> Inserting word, definition, example, synonyms

    conn.commit() 

In [18]:
insert_data_into_db(all_word_entries)  # ---> Insert all words into database


In [19]:
cursor.execute("SELECT * FROM words LIMIT 10")  # ---> Fetch first 10 rows
print(cursor.fetchall())  # ---> Print results


[('breathe', 'draw air into, and expel out of, the lungs; "I can breathe better when the air is clean"; "The patient is respiring"', None, 'respire, suspire'), ('respire', 'undergo the biomedical and metabolic processes of respiration by taking up oxygen and producing carbon monoxide', None, None), ('respire', 'breathe easily again, as after exertion or anxiety', None, None), ('choke', 'breathe with great difficulty, as when experiencing a strong emotion; "She choked with emotion when she spoke about her deceased husband"', None, None), ('hyperventilate', 'breathe excessively hard and fast; "The mountain climber started to hyperventilate"', None, None), ('hyperventilate', 'produce hyperventilation in; "The nurses had to hyperventilate the patient"', None, None), ('aspirate', 'inhale (air, water, etc.)', None, None), ('burp', 'expel gas from the stomach; "Please don\'t burp at the table"', None, 'bubble, belch, eruct'), ('hiccup', 'breathe spasmodically, and make a sound; "When you have

In [20]:
conn.close()  # ---> Close database connection
